In [1]:
!pip install langchain
!pip install llama_index
import constants.constants as constants
import prompts.prompt_classifier as prompt_classifier
import prompts.prompt_banking as prompt_banking
import prompts.prompt_cards as prompt_card
import prompts.prompt_oeffnungszeiten as prompt_oeffnungszeiten
import functions.functions as functions

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType

import openai
import os 
os.environ["OPENAI_API_KEY"] = constants.OPENAI_API_KEY
import llama_index
import json
from llama_index import download_loader
from pathlib import Path
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

In [2]:
chat = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY)


@tool
def debit_card_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren von Bankomat-, Debit- und Kreditkarten."""
    messages = [
        SystemMessage(content=prompt_card.prompt_cards),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)
@tool
def banking_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren des Online-Bankings."""
    messages = [
        SystemMessage(content=prompt_banking.prompt_banking),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)



In [3]:
SimpleCSVReader = download_loader("SimpleCSVReader")
loader = SimpleCSVReader(concat_rows=False)
documents = loader.load_data(file=Path('prompts/prompt_oeffnungszeiten.py'))
index = GPTVectorStoreIndex.from_documents(documents)

In [4]:
class Chatbot:
    def __init__(self, api_key, index):
        self.index = index
        openai.api_key = api_key
        self.chat_history = []

    def generate_response(self, user_input):
        prompt = "\n".join([f"{message['role']}: {message['content']}" 
                           for message in self.chat_history[-5:]])
        prompt += f"\nUser: {user_input}"
        query_engine = index.as_query_engine()
        response = query_engine.query(user_input)

        message = {"role": "Du bist ein deutscher assistent, der auf Deutsch antwortet! ", "content": response.response}
        self.chat_history.append({"role": "user", "content": user_input})
        self.chat_history.append(message)
        return message

    def load_chat_history(self, filename):
        try:
            with open(filename, 'r') as f:
                self.chat_history = json.load(f)
        except FileNotFoundError:
            pass

    def save_chat_history(self, filename):
        with open(filename, 'w') as f:
            json.dump(self.chat_history, f)

bot = Chatbot("Add your API Key here ", index=index)
bot.load_chat_history("chat_history.json")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["bye", "goodbye"]:
        print("Bot: Goodbye!")
        bot.save_chat_history("chat_history.json")
        break
    response = bot.generate_response(user_input)
    print(f"Bot: {response['content']}")
    break


In [5]:
#USE CASE ÖFFNUNGSZEITEN
@tool
def banking_oeffnungszeiten(question):
    """Beantwortet Fragen zu dem Thema: Öffnungszeiten von Banken."""
    bot = Chatbot("Add your API Key here ", index=index)
    response = bot.generate_response(question)

    return f"Bot: {response['content']}"


In [6]:
tools = [debit_card_information, banking_information, banking_oeffnungszeiten]
messages = [
    SystemMessage(content=prompt_classifier.system_message),
    first_ai_msg := AIMessage(content="Hier ist der Chatbot der Raiffeisenlandesbank Niederösterreich-Wien. Gerne beantworte ich Ihre Fragen zur Sperre von Debit- und Kreditkarten sowie zur Sperre des Online-Bankings."),
]


print(first_ai_msg.content)
question_of_user = str(input(""))
messages.append(HumanMessage(content=question_of_user))


mrkl = initialize_agent(tools, chat, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
print(mrkl.run(messages))


Hier ist der Chatbot der Raiffeisenlandesbank Niederösterreich-Wien. Gerne beantworte ich Ihre Fragen zur Sperre von Debit- und Kreditkarten sowie zur Sperre des Online-Bankings.


 Öffnungszeiten Andromeda-Tower, Donau City Str. 6




> Entering new AgentExecutor chain...

Invoking: `banking_oeffnungszeiten` with `{'question': 'Was sind die Öffnungszeiten des Andromeda-Towers in der Donau City Str. 6?'}`


Bot: Die Öffnungszeiten des Andromeda-Towers in der Donau City Str. 6 sind von 08:00 bis 18:00 Uhr.Die Öffnungszeiten des Andromeda-Towers in der Donau City Str. 6 sind von 08:00 bis 18:00 Uhr.

> Finished chain.
Die Öffnungszeiten des Andromeda-Towers in der Donau City Str. 6 sind von 08:00 bis 18:00 Uhr.
